In [1]:
import math
import random

## 2.1.   Modeling the queen’s egg laying behavior

In [2]:
def season(t):
    # (1)
    return max(1-(1/(1+x1*math.exp(-2*t/x2))),1/(1+x3*math.exp(-2*(t-x4)/x5)))

In [3]:
def CELLSempty(t):
    # (2)
    return CELLShive - CELLSbrood(t) - STORESpollen(t) - STORESnectar(t) - STOREShoney(t)

In [4]:
def SUPcomb(t):
    # (3)
    if CELLSempty(t)/(CELLShive + 1) < SUPthreshold:
        return CELLSempty(t)/((CELLShive + 1) * SUPthreshold)
    else:
        return 1

In [5]:
def ELRstoch(t):
    # (4)
    return random.uniform(-ELRstochrange,+ELRstochrange)

In [6]:
def ELR(t):
    # (5)
    return ELRbase * (1 + ELRstoch(t)) * (1 - season(t)) * SUPcomb(t) 

## 2.2. Modeling the immature stages

### 2.2.1 Eggs

In [7]:
def EGGS(i,t):
    # (6)
    if i == 1:
        return ELR(t-1) * (1 - MORTALITYeggs)
    elif i in [2,3]:
        return EGGS(i-1,t-1) * (1 - MORTALITYeggs) 

In [8]:
def CELLSeggs(t):
    # (7)
    sum = 0
    for i in range(LIFESPANegg+1):
        if i != 0:
            sum += EGGS((i,t))
    return sum

In [9]:
def CELLSeggs(t):
    # (8)
    # alternative to (7)
    sum = 0
    for i in range(LIFESPANegg+1):
        if i != 0:
            sum += (ELR(t-i)*math.pow(1-MORTALITYeggs,i))
    return sum

### 2.2.2. Larvae (unsealed)

In [10]:
def SURVIVALlarvae(i,t):
    # (9)
    return (1 - CANNIBALISMlarvae(i,t)) * (1 - MORTALITYlarvae)

In [11]:
def CANNIBALISMlarvae(i,t):
    # (10)
    return CANNIBALISMhungerbase(i) * (1 - (INDEXpollensituation(t-1) * INDEXnursingquality(t-1)))

In [12]:
def LARVAE(i,t):
    # (11)
    if i == 1:
        return EGGS(LIFESPANegg,t-1)*SURVIVALlarvae(1,t)
    elif 1 < i and i <= LIFESPANlarva:
        return LARVAE(i-1,t-1) * SURVIVALlarvae(i,t)

In [13]:
def CELLSlarvae(t):
    # (12)
    sum = 0
    for i in range(LIFESPANlarva+1):
        if i != 0:
            sum += LARVAE((i,t))
    return sum 

In [14]:
def CELLSlarvae(t):
    # (13)
    # alternative to (12)
    
    def product(i):
        prdct = 1
        for k in range(i+1):
            if k != 0:
                prdct *= SURVIVALlarvae(k,t-i+k)
        return prdct
    
    sum = 0
    for i in range(LIFESPANlarva+1):
        if i != 0:
            sum += (ELR(t-i-LIFESPANegg)*math.pow(1-MORTALITYeggs,i)*product(i))  
    
    return sum

In [15]:
def PUPAE(i,t):
    # (14)
    if i == 1:
        return LARVAE(LIFESPANlarva,t-1) * (1 - MORTALITYpupae)
    elif 1 < i and i <= LIFESPANpupa:
        return PUPAE(i-1,t-1) * (1 - MORTALITYpupae)

In [16]:
def CELLSpupae(t):
    # (15)
    sum = 0
    for i in range(LIFESPANpupa+1):
        if i != 0:
            sum += PUPAE((i,t))
    return sum 

In [17]:
def CELLSpupae(t):
    # (16)
    # alternative to (15)
    
    def product(i):
        prdct = 1
        for k in range(LIFESPANlarva+1):
            if k != 0:
                prdct *= SURVIVALlarvae(k,t-i-LIFESPANlarva+k)
        return prdct
    
    sum = 0
    for i in range(LIFESPANpupa+1):
        if i != 0:
            sum += (ELR(t-i-LIFESPANegg-LIFESPANlarva)*math.pow(1-MORTALITYeggs,LIFESPANegg)*product(i)*math.pow(1-MORTALITYpupae,i))  
    
    return sum

In [18]:
def CELLSbrood(t):
    # (17)
    return CELLSeggs(t) + CELLSlarvae(t) + CELLSpupae(t)